In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/diabetes.csv")
df.shape

(768, 9)

In [ ]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_scaled

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y,test_size=0.2,random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [ ]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation="sigmoid"))

In [ ]:
model.compile(loss = "binary_crossentropy",optimizer='Adam',metrics=["accuracy"])

In [ ]:
model.fit(x_train,y_train,batch_size=32,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
20/20 [==============================] - 4s 52ms/step - loss: 0.7513 - accuracy: 0.4381 - val_loss: 0.6907 - val_accuracy: 0.5455
Epoch 2/100
20/20 [==============================] - 0s 10ms/step - loss: 0.6862 - accuracy: 0.5570 - val_loss: 0.6500 - val_accuracy: 0.6429
Epoch 3/100
20/20 [==============================] - 0s 8ms/step - loss: 0.6401 - accuracy: 0.6336 - val_loss: 0.6171 - val_accuracy: 0.6818
Epoch 4/100
20/20 [==============================] - 0s 9ms/step - loss: 0.6032 - accuracy: 0.6873 - val_loss: 0.5955 - val_accuracy: 0.7208
Epoch 5/100
20/20 [==============================] - 0s 8ms/step - loss: 0.5763 - accuracy: 0.7280 - val_loss: 0.5766 - val_accuracy: 0.7273
Epoch 6/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5538 - accuracy: 0.7508 - val_loss: 0.5620 - val_accuracy: 0.7338
Epoch 7/100
20/20 [==============================] - 0s 10ms/step - loss: 0.5362 - accuracy: 0.7524 - val_loss: 0.5500 - val_accuracy: 0.7338
Epoch 8/10

In [ ]:
pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.7 MB/s eta 0:00:00


In [ ]:
import kerastuner as kt

<ipython-input-14-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [ ]:
# automating to find the best optimizer using the keras tuner

def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizers = hp.Choice("optimizer",values = ['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizers,loss = "binary_crossentropy",metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 3)

In [ ]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.5454545617103577

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 00m 08s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train,y_train,epochs=100,batch_size=32,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 19ms/step - loss: 0.5134 - accuracy: 0.7557 - val_loss: 0.5206 - val_accuracy: 0.7662
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5022 - accuracy: 0.7638 - val_loss: 0.5131 - val_accuracy: 0.7597
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4937 - accuracy: 0.7655 - val_loss: 0.5085 - val_accuracy: 0.7662
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4864 - accuracy: 0.7736 - val_loss: 0.5067 - val_accuracy: 0.7597
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4798 - accuracy: 0.7720 - val_loss: 0.5065 - val_accuracy: 0.7597
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4752 - accuracy: 0.7736 - val_loss: 0.5045 - val_accuracy: 0.7662
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4711 - accuracy: 0.7752 - val_loss: 0.5028 - val_accuracy: 0.7662
Epoch 14

In [ ]:
# automating to find the number of neurons using the keras tuner

def build_model(hp):

  model = Sequential()

  units = hp.Int('Neurons',8,128,step=8)

  model.add(Dense(units,activation='relu',input_dim=8))
  model.add(Dense(1,activation="sigmoid"))

  model.compile(optimizer='rmsprop',loss="binary_crossentropy",metrics=['accuracy'])
  
  return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name='soorya ram')

In [ ]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 15s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'Neurons': 72}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 1s 3ms/step - loss: 0.4860 - accuracy: 0.7590
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4749 - accuracy: 0.7736
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4659 - accuracy: 0.7720
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4590 - accuracy: 0.7720
Epoch 11/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4542 - accuracy: 0.7752
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4508 - accuracy: 0.7671
Epoch 13/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4471 - accuracy: 0.7769
Epoch 14/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4449 - accuracy: 0.7752
Epoch 15/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4420 - accuracy: 0.7834
Epoch 16/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4399 - accuracy: 0.7850
Epo

In [ ]:
# automating to find the number of neurons using the keras tuner

def build_model(hp):

  model= Sequential()
  model.add(Dense(72,activation='relu',input_dim=8))

  for i in range(hp.Int("Layers_decision",min_value=1,max_value=10)):
    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  
  return model


In [ ]:
tuner = kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=3,
                     directory='mydir',
                     project_name='num_layers')

In [ ]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

{'Layers_decision': 2}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 1s 3ms/step - loss: 0.4453 - accuracy: 0.7801
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4346 - accuracy: 0.7834
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4283 - accuracy: 0.8062
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4193 - accuracy: 0.7964
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4141 - accuracy: 0.8143
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4092 - accuracy: 0.8094
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4073 - accuracy: 0.8029
Epoch 14/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4044 - accuracy: 0.8094
Epoch 15/100
20/20 [==============================] - 0s 3ms/step - loss: 0.3962 - accuracy: 0.8062
Epoch 16/100
20/20 [==============================] - 0s 3ms/step - loss: 0.3909 - accuracy: 0.8160
Epo

In [ ]:
# combining all the strategies leart above 
def build_model(hp):
  
  model = Sequential()
  
  counter = 0
  
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter == 0:
      model.add(Dense(
          hp.Int("neuron"+str(i),min_value=8,max_value=128,step=8),
          activation = hp.Choice("activation"+str(i),values=['relu','sigmoid','tanh']),
          input_dim=8
      ))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int("neuron"+str(i),min_value=8,max_value=128,step=8),
                      activation=hp.Choice('activation'+str(i),values=['relu',"sigmoid",'tanh'])))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter +=1
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice("optimizer",values=['adam','sgd','rmsprop','adadelta']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
  
  return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='combined1',
                        project_name='sooryaRamakrishna')

In [ ]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 10s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8,
 'neuron0': 8,
 'activation0': 'relu',
 'dropout0': 0.1,
 'optimizer': 'sgd',
 'neuron1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'neuron2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'neuron3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'neuron4': 8,
 'activation4': 'relu',
 'dropout4': 0.1,
 'neuron5': 8,
 'activation5': 'relu',
 'dropout5': 0.1,
 'neuron6': 8,
 'activation6': 'relu',
 'dropout6': 0.1,
 'neuron7': 8,
 'activation7': 'relu',
 'dropout7': 0.1}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]
model.fit(x_train,y_train,epochs=200,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/200
20/20 [==============================] - 2s 20ms/step - loss: 0.6813 - accuracy: 0.6221 - val_loss: 0.6742 - val_accuracy: 0.6429
Epoch 8/200
20/20 [==============================] - 0s 5ms/step - loss: 0.6712 - accuracy: 0.6498 - val_loss: 0.6689 - val_accuracy: 0.6429
Epoch 9/200
20/20 [==============================] - 0s 5ms/step - loss: 0.6664 - accuracy: 0.6482 - val_loss: 0.6650 - val_accuracy: 0.6429
Epoch 10/200
20/20 [==============================] - 0s 5ms/step - loss: 0.6637 - accuracy: 0.6515 - val_loss: 0.6614 - val_accuracy: 0.6429
Epoch 11/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6583 - accuracy: 0.6531 - val_loss: 0.6582 - val_accuracy: 0.6429
Epoch 12/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6536 - accuracy: 0.6531 - val_loss: 0.6559 - val_accuracy: 0.6429
Epoch 13/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6540 - accuracy: 0.6531 - val_loss: 0.6543 - val_accuracy: 0.6429
Epoch 14